# LLM Inference with Burn on Google Colab

このノートブックは、Burn MLフレームワークを使用してGoogle Colab上でLLM推論を実行します。

**対応GPU:**
- Tesla T4 (無料版)
- A100 (Colab Pro+)
- H100 (Colab Enterprise)

**必要な設定:**
- ランタイムタイプ: GPU
- GPU種類: T4, A100, またはH100

## 1. GPU確認

In [10]:
!nvidia-smi

Sun Jan 18 12:59:13 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2. Rustのインストール

In [ ]:
# Rustのインストール
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y

# 環境変数の設定
import os
os.environ['PATH'] = f"/root/.cargo/bin:{os.environ['PATH']}"

# インストール確認
!rustc --version
!cargo --version

## 3. CUDA開発環境の確認

In [11]:
# CUDAバージョン確認
!nvcc --version

# CUDA環境変数の設定
!echo $CUDA_HOME
!echo $LD_LIBRARY_PATH

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0

/usr/lib64-nvidia


## 4. プロジェクトのクローン/アップロード

In [14]:
# 方法1: GitHubからクローン (リポジトリがある場合)
!git clone https://github.com/Tatsuya0518/LLM-API.git
# %cd LLM-API

# 方法2: ローカルファイルをアップロード
# 1. 左サイドバーのファイルアイコンをクリック
# 2. LLM-APIフォルダ全体をドラッグ&ドロップ
# 3. 以下のセルを実行してディレクトリに移動

%cd /content/LLM-API

Cloning into 'LLM-API'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 26 (delta 2), reused 26 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 18.83 KiB | 18.83 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/LLM-API


## 5. 依存関係のビルド

In [15]:
# Cargo.tomlの確認
!cat Cargo.toml

[package]
name = "llm-api"
version = "0.1.0"
edition = "2021"

[dependencies]
# Burn ML Framework with CUDA backend for NVIDIA GPUs (T4, H100, A100)
burn = { version = "0.15", features = ["train", "dataset"] }
burn-cuda = "0.15"

# Tensor operations (for CPU fallback and testing)
burn-ndarray = "0.15"

# Tokenization
tokenizers = { version = "0.20", default-features = false, features = ["onig"] }

# Serialization
serde = { version = "1.0", features = ["derive"] }
serde_json = "1.0"

# Async runtime
tokio = { version = "1", features = ["full"] }

# Logging
tracing = "0.1"
tracing-subscriber = "0.3"

# Utilities
anyhow = "1.0"
rand = "0.8"

[profile.release]
opt-level = 3
lto = true
codegen-units = 1


In [ ]:
# ビルド (初回は10-20分かかります)
!cargo build --release

## 6. 推論の実行

In [ ]:
# 推論実行
!cargo run --release

## 7. カスタムプロンプトで推論

In [ ]:
# カスタムプロンプトを使用する場合は、main.rsを編集してください
# または、CLIオプションを追加する実装が必要です

## トラブルシューティング

### CUDAが見つからない場合
```bash
export CUDA_HOME=/usr/local/cuda
export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH
export PATH=/usr/local/cuda/bin:$PATH
```

### メモリ不足の場合
- モデルサイズを小さくする（`ModelConfig::tiny()`を使用）
- より大きなGPU（A100やH100）を使用

### ビルドエラーの場合
```bash
cargo clean
cargo build --release
```